# Post Processing

This notebook run some post processing on result point cloud. 
1. Convert point cloud to DEM with `point2dem` command from ASP.
2. Coregister with reference DEMs.
3. Compute difference between coregistered DEMs and references DEMs.
4. Compute some statistics of this difference.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import history

/workdir/cryodyn/dehecqa/bin/miniforge3/envs/history/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Settings

In [11]:
# contain the .las or .laz files
POINTCLOUD_DIRECTORY = "/mnt/summer/USERS/DEHECQA/history/output/test_data"

# Directory where put DEMs after point2dem command
RAW_DEM_DIRECTORY = "/mnt/summer/USERS/DEHECQA/history/output/test_data/dems"

# Directory of coregistered DEMs
COREGISTERED_DEM_DIRECTORY = "/mnt/summer/USERS/DEHECQA/history/output/test_data/coregistered_dems"

# Directory of diference beetween ref DEMs and coregistered DEMs
DDEMS_DIRECTORY = "/mnt/summer/USERS/DEHECQA/history/output/test_data/ddems"

# dictionnary of all references DEMs (some value can be set to None depend on site and dataset)
REF_DEMS = {
    "casagrande_ref_dem_zoom" : "/mnt/summer/USERS/DEHECQA/history/data_final/casa_grande/aux_data/reference_dem_zoom.tif",
    "casagrande_ref_dem_large" : None,
    "iceland_ref_dem_zoom" : None,
    "iceland_ref_dem_large" : None
}

# dictionnary of all references DEMs masks (some value can be set to None depend on site and dataset)
REF_DEM_MASKS = {
    "casagrande_ref_dem_zoom_mask" : "/mnt/summer/USERS/DEHECQA/history/data_final/casa_grande/aux_data/reference_dem_zoom_mask.tif",
    "casagrande_ref_dem_large_mask" : None,
    "iceland_ref_dem_zoom_mask" : None,
    "iceland_ref_dem_large_mask" : None
}

OVERWRITE = False

# set this to True to avoid process
DRY_RUN = False

MAX_WORKERS = 4

## Converting cloud point to DEM

Batch process point cloud files in a directory to generate DEMs aligned with reference DEMs.

This function iterates over all point cloud files (*.las or *.laz) in `input_directory`,
selects the appropriate reference DEM based on site and dataset extracted from filenames,
and calls `point2dem` to create coregistered DEMs saved in `output_directory`.

The `max_workers` arg set a process for each point2dem command so for one file it's useless

Different attempts to add ASP to path, without success...

In [8]:
# !module load stereo3.5.0
# !export PATH=$PATH:/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/
# !point2dem -h
# !echo $PATH

In [9]:
# import subprocess
# subprocess.run("export PATH=$PATH:/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/", shell=True, check=True)
# subprocess.run("echo $PATH", shell=True, check=True)

In [12]:
history.postprocessing.iter_point2dem(POINTCLOUD_DIRECTORY, RAW_DEM_DIRECTORY, **REF_DEMS, overwrite=OVERWRITE, dry_run=DRY_RUN, max_workers=MAX_WORKERS, asp_path="/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/")

Skip ASPY_CG_MC_PP_CN_GN_PN_MN_dense_pointcloud.laz : No reference DEM provided (site: CG, dataset: MC).
Skip ESM_IL_AI_PP_CN_GY_PN_MN_dense_pointcloud.laz : No reference DEM provided (site: IL, dataset: AI).
Skip ESM_IL_AI_RA_CN_GY_PN_MN_dense_pointcloud.laz : No reference DEM provided (site: IL, dataset: AI).
Skip ESM_IL_AI_RA_CY_GY_PN_MN_dense_pointcloud.laz : No reference DEM provided (site: IL, dataset: AI).
Skip TUW_IL_AI_PP_CN_GN_PN_MN_dense_pointcloud.laz : No reference DEM provided (site: IL, dataset: AI).
Skip HSfM_IL_AI_PP_CY_GY_PN_MN_dense_pointcloud.laz : No reference DEM provided (site: IL, dataset: AI).


Converting into DEM:  30%|███       | 3/10 [36:24<1:20:20, 688.66s/File] 

[!] Error: Command '/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/point2dem --t_srs "+proj=utm +zone=12 +datum=NAD83 +units=m +no_defs=True" --tr 1.0 --t_projwin 413999.9999714241 3613020.0000490383 443999.9999714241 3650006.0000490383 --threads 1 --datum WGS84  /mnt/summer/USERS/DEHECQA/history/output/test_data/fdahle_CG_AI_PP_CN_GY_PN_MN_dense_pointcloud.laz -o /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/fdahle_CG_AI_PP_CN_GY_PN_MN_dense' returned non-zero exit status 1.


Converting into DEM:  40%|████      | 4/10 [36:25<41:43, 417.23s/File]  

[!] Error: Command '/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/point2dem --t_srs "+proj=utm +zone=12 +datum=NAD83 +units=m +no_defs=True" --tr 1.0 --t_projwin 413999.9999714241 3613020.0000490383 443999.9999714241 3650006.0000490383 --threads 1 --datum WGS84  /mnt/summer/USERS/DEHECQA/history/output/test_data/fdahle_CG_AI_PP_CY_GN_PN_MN_dense_pointcloud.laz -o /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/fdahle_CG_AI_PP_CY_GN_PN_MN_dense' returned non-zero exit status 1.


Converting into DEM:  50%|█████     | 5/10 [36:27<22:17, 267.41s/File]

[!] Error: Command '/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/point2dem --t_srs "+proj=utm +zone=12 +datum=NAD83 +units=m +no_defs=True" --tr 1.0 --t_projwin 413999.9999714241 3613020.0000490383 443999.9999714241 3650006.0000490383 --threads 1 --datum WGS84  /mnt/summer/USERS/DEHECQA/history/output/test_data/fdahle_CG_AI_PP_CY_GY_PN_MN_dense_pointcloud.laz -o /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/fdahle_CG_AI_PP_CY_GY_PN_MN_dense' returned non-zero exit status 1.


Converting into DEM:  60%|██████    | 6/10 [36:28<11:48, 177.00s/File]

[!] Error: Command '/workdir2/cryodyn/dehecqa/bin/StereoPipeline-3.5.0-alpha-2025-02-18-x86_64-Linux/bin/point2dem --t_srs "+proj=utm +zone=12 +datum=NAD83 +units=m +no_defs=True" --tr 1.0 --t_projwin 413999.9999714241 3613020.0000490383 443999.9999714241 3650006.0000490383 --threads 1 --datum WGS84  /mnt/summer/USERS/DEHECQA/history/output/test_data/TUW_CG_AI_PP_CN_GN_PN_MN_dense_pointcloud.laz -o /mnt/summer/USERS/DEHECQA/history/output/test_data/dems/TUW_CG_AI_PP_CN_GN_PN_MN_dense' returned non-zero exit status 1.


Converting into DEM: 100%|██████████| 10/10 [1:15:35<00:00, 453.54s/File]


## Coregister DEMs

Coregister multiple DEMs in a directory to appropriate reference DEMs and return summary statistics.

This function iterates over all DEM files in `input_directory` ending with '-DEM.tif', 
selects the appropriate reference DEM and mask based on the site and dataset information,
and applies coregistration using the `coregister_dem` function. 

Coregistered DEMs are saved to `output_directory`. If `overwrite` is False, existing output files are skipped.
If `dry_run` is True, no coregistration is performed, only file names and planned operations are printed.

In [ ]:
result_df = history.postprocessing.iter_coregister_dems(
    RAW_DEM_DIRECTORY, COREGISTERED_DEM_DIRECTORY, 
    **REF_DEMS, **REF_DEM_MASKS, overwrite=OVERWRITE, dry_run=DRY_RUN, output_ddem_after_directory=DDEMS_DIRECTORY
)
result_df

coregister_dem(/mnt/summer/USERS/DEHECQA/history/output/test_data/dems/HSfM_CG_AI_PP_CY_GY_PY_MN_dense-DEM.tif, /mnt/summer/USERS/DEHECQA/history/data_final/casa_grande/aux_data/reference_dem_zoom.tif, /mnt/summer/USERS/DEHECQA/history/data_final/casa_grande/aux_data/reference_dem_zoom_mask.tif, /mnt/summer/USERS/DEHECQA/history/output/test_data/coregistered_dems/HSfM_CG_AI_PP_CY_GY_PY_MN_dense-DEM.tif)
